# SCRAPING DATA FROM FIBA INCLUDING, PLAYER NAME, IF SHOT WAS MADE OR NOT, THE POSITION SHOT WAS ATTEMPTED, TEAM AND OPPONENT

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup

import time

# from requests_html import AsyncHTMLSession, HTMLSession

import pandas as pd

import re


In [2]:
chrome_options = webdriver.ChromeOptions()

chrome_options.binary_location = 'C:/chromedriver/chrome-win64/chrome.exe'
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument('--start-maximized')

In [3]:
service = Service('C:/chromedriver/chromedriver-win64/chromedriver.exe')

In [4]:
url = 'https://www.fiba.basketball/en/events/fiba-u17-basketball-world-cup-2024/games/122637-AUS-PHI#shotChart'

In [5]:
driver = webdriver.Chrome(service=service,options=chrome_options)

driver.get(url)

game = driver.title

# Scroll to the middle of the page
time.sleep(3)
# driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
# time.sleep(2)

svg_court_element = driver.find_element(By.CSS_SELECTOR, 'div._1xyado84 svg')
court_width = svg_court_element.get_attribute('width')
court_height = svg_court_element.get_attribute('height')

driver.execute_script("window.scrollTo(0, 650);")
time.sleep(2)

# Set zoom to 70%
# driver.execute_script("document.body.style.zoom = '-50';")
# driver.execute_script("document.body.style.zoom='80%';")
# time.sleep(2)
# Wait for the elements to be interactable

wait = WebDriverWait(driver, 10)

uncheck_home_all = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[3]/div[7]/div/div/div[1]/div/main/div/div/div/div[4]/div[1]/div')
uncheck_home_all.click()

uncheck_away_all = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[3]/div[7]/div/div/div[1]/div/main/div/div/div/div[4]/div[2]/div')
uncheck_away_all.click()

red_made = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[3]/div[7]/div/div/div[1]/div/main/div/div/div/div[3]/div[1]/div[1]')
red_made.click()

red_missed = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[3]/div[7]/div/div/div[1]/div/main/div/div/div/div[3]/div[1]/div[2]')
red_missed.click()

blue_made = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[3]/div[7]/div/div/div[1]/div/main/div/div/div/div[3]/div[2]/div[1]')
blue_made.click()

blue_missed = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[3]/div[7]/div/div/div[1]/div/main/div/div/div/div[3]/div[2]/div[2]')
blue_missed.click()

time.sleep(3)

player_names_div = driver.find_elements(By.CLASS_NAME, '_1xyado8i')
player_names = [name.text for name in player_names_div]

all_names = []
red_team_names = []
blue_team_names = []

for i in player_names:
    for k in i.split('\n'):
        if len(k) > 3:
            all_names.append(k)

#split names to groups
for i in range(len(all_names)):
    if i%2 != 0:
        blue_team_names.append(all_names[i])
    else:
        red_team_names.append(all_names[i])
        
teams = game.split()[:3]


def get_player_position(_x, _y):
    # FIBA court dimensions in meters
    court_width = 28
    court_height = 15
    
    # Viewbox dimensions in pixels
    viewbox_width = court_width
    viewbox_height = court_height
    
    # Player's position in percentage
    player_x_percent = _x
    player_y_percent = _y
    
    # Convert percentage to pixel values in the viewbox
    player_x_pixel = (player_x_percent / 100) * viewbox_width
    player_y_pixel = (player_y_percent / 100) * viewbox_height
    
    # Map pixel values to FIBA court dimensions
    player_x_court = (player_x_pixel / viewbox_width) * court_width
    player_y_court = (player_y_pixel / viewbox_height) * court_height
    
    return player_x_court, player_y_court

Team = []
Player_Name = []
Opponent = []
x_y_coordinate = []
Shot = []

def run_shot_chart(shot):
    for name in all_names:
        _name = wait.until(EC.presence_of_element_located((By.XPATH, f"//div[contains(text(), '{name}')]")))
        _name.click()
        time.sleep(3)

        shot_spots = driver.find_elements(By.CLASS_NAME, '_1xyado86')

        if len(shot_spots) > 0:
            for i in shot_spots:
                style = i.get_attribute('style')
                x_coord = re.search(r'--_1xyado80:\s*([\d.]+)%', style)
                y_coord = re.search(r'--_1xyado81:\s*([\d.]+)%', style)
                
                if name in red_team_names:
                    print(f'Team: {teams[0]}')
                    Team.append(teams[0])
                else:
                    print(f'Team: {teams[2]}')
                    Team.append(teams[2])
                    
                print(f'Player: {name}')
                Player_Name.append(name)
                                    
                if name in blue_team_names:
                    print(f'Opponent: {teams[0]}')
                    Opponent.append(teams[0])
                else:
                    print(f'Opponent: {teams[2]}')
                    Opponent.append(teams[2])
                    
                pos_ = get_player_position(float(x_coord.group(1)), float(y_coord.group(1)))    
                print(f'x coordinate = {pos_[0]}')
                print(f'y coordinate = {pos_[1]}')
                x_y_coordinate.append(f'{pos_[0]}, {pos_[1]}')
                
                print(f'Status: {shot}')
                Shot.append(shot)
                
                print('++++++++++++++++++++++++++++++++++++')

        _name.click()

for i in [1,2]:
    if i == 1:
        red_made.click()
        blue_made.click()
        time.sleep(3)
        run_shot_chart('made')
        driver.execute_script("window.scrollTo(0, 650);")
        red_made.click()
        blue_made.click()
        time.sleep(2)
    else:
        time.sleep(2)
        red_missed.click()
        blue_missed.click()
        time.sleep(2)
        run_shot_chart('missed')

# create dataframe
dict_for_df = {'Team': Team, 'Player': Player_Name, 'Opponent': Opponent, 'x_y_position': x_y_coordinate, 'Shot': Shot}
final_dataframe = pd.DataFrame(dict_for_df)

time.sleep(2)

driver.quit()


Team: Australia
Player: N. Walker
Opponent: Philippines
x coordinate = 8.099616858237548
y coordinate = 4.5
Status: made
++++++++++++++++++++++++++++++++++++
Team: Australia
Player: N. Walker
Opponent: Philippines
x coordinate = 1.8773946360153255
y coordinate = 7.982142857142857
Status: made
++++++++++++++++++++++++++++++++++++
Team: Australia
Player: N. Walker
Opponent: Philippines
x coordinate = 5.149425287356322
y coordinate = 1.446428571428571
Status: made
++++++++++++++++++++++++++++++++++++
Team: Australia
Player: N. Walker
Opponent: Philippines
x coordinate = 1.770114942528736
y coordinate = 7.339285714285715
Status: made
++++++++++++++++++++++++++++++++++++
Team: Australia
Player: N. Walker
Opponent: Philippines
x coordinate = 7.616858237547893
y coordinate = 3.6428571428571437
Status: made
++++++++++++++++++++++++++++++++++++
Team: Australia
Player: N. Walker
Opponent: Philippines
x coordinate = 1.8237547892720305
y coordinate = 6.857142857142857
Status: made
++++++++++++++++

In [6]:
final_dataframe.sort_values(by=['Player'])

,Team,Player,Opponent,x_y_position,Shot
27,Australia,A. Dickeson,Philippines,"0.5900383141762452, 0.42857142857142916",made
29,Australia,A. Dickeson,Philippines,"1.3946360153256705, 6.857142857142857",made
30,Australia,A. Dickeson,Philippines,"1.8237547892720305, 8.357142857142858",made
31,Australia,A. Dickeson,Philippines,"2.091954022988506, 8.303571428571427",made
100,Australia,A. Dickeson,Philippines,"7.3486590038314175, 12.0",missed
...,...,...,...,...,...
46,Australia,S. Pilimai,Philippines,"1.8773946360153255, 6.589285714285714",made
125,Australia,S. Pilimai,Philippines,"1.7164750957854404, 6.32142857142857",missed
124,Australia,S. Pilimai,Philippines,"1.8237547892720305, 4.7678571428571415",missed
123,Australia,S. Pilimai,Philippines,"1.6091954022988506, 8.142857142857142",missed


In [7]:
## SAVE TO CSV

In [8]:
final_dataframe.to_csv('fiba_aus_phil_details.csv', encoding='utf-8-sig', index=False)

In [9]:
data = pd.read_csv('fiba_aus_phil_details.csv')

In [10]:
data

,Team,Player,Opponent,x_y_position,Shot
0,Australia,N. Walker,Philippines,"8.099616858237548, 4.5",made
1,Australia,N. Walker,Philippines,"1.8773946360153255, 7.982142857142857",made
2,Australia,N. Walker,Philippines,"5.149425287356322, 1.446428571428571",made
3,Australia,N. Walker,Philippines,"1.770114942528736, 7.339285714285715",made
4,Australia,N. Walker,Philippines,"7.616858237547893, 3.6428571428571437",made
...,...,...,...,...,...
146,Philippines,E. Morales,Australia,"20.27586206896552, 10.660714285714286",missed
147,Philippines,E. Morales,Australia,"20.758620689655174, 3.321428571428571",missed
148,Australia,R. Akhuar,Philippines,"1.6091954022988506, 8.357142857142858",missed
149,Australia,R. Akhuar,Philippines,"3.2720306513409962, 7.446428571428571",missed
